# Coffee Shop Sales Regression

1. Carga y previsualización del dataset (`.xlsx`).
2. Limpieza y extracción de features (fecha/hora → variables numéricas).
3. Cálculo de ventas por transacción y agrupación horaria por tienda.
4. Creación y entrenamiento de un modelo LinearRegression.
5. Predicción y comparación con ventas reales.
6. Gráficos en Matplotlib que comparan real vs. predicho.
7. Guardado de resultados finales.


---

Cargamos y previsualizamos el dataset

- Cree una variable por si necesito cambiar el dataset o si cambio de nombre al dataset.
- Con read leemos el dataset que tiene formato excel
- Se muestra tambien las primeras filas, dimensiones y nombres de las columnas

In [ ]:
import pandas as pd

file_path = 'Coffee-Shop-Sales.xlsx'

df = pd.read_excel(file_path)

print('\nShape:', df.shape)
print('Columns:', df.columns.tolist())
df.head()


Shape: (149116, 11)
Columns: ['transaction_id', 'transaction_date', 'transaction_time', 'transaction_qty', 'store_id', 'store_location', 'product_id', 'unit_price', 'product_category', 'product_type', 'product_detail']


,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail
0,1,2023-01-01,07:06:11,2,5,Lower Manhattan,32,3.0,Coffee,Gourmet brewed coffee,Ethiopia Rg
1,2,2023-01-01,07:08:56,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg
2,3,2023-01-01,07:14:04,2,5,Lower Manhattan,59,4.5,Drinking Chocolate,Hot chocolate,Dark chocolate Lg
3,4,2023-01-01,07:20:24,1,5,Lower Manhattan,22,2.0,Coffee,Drip coffee,Our Old Time Diner Blend Sm
4,5,2023-01-01,07:22:41,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg


## Analizamos las columnas

In [12]:
df.info()

df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149116 entries, 0 to 149115
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   transaction_id    149116 non-null  int64         
 1   transaction_date  149116 non-null  datetime64[ns]
 2   transaction_time  149116 non-null  object        
 3   transaction_qty   149116 non-null  int64         
 4   store_id          149116 non-null  int64         
 5   store_location    149116 non-null  object        
 6   product_id        149116 non-null  int64         
 7   unit_price        149116 non-null  float64       
 8   product_category  149116 non-null  object        
 9   product_type      149116 non-null  object        
 10  product_detail    149116 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(4), object(5)
memory usage: 12.5+ MB


transaction_id      0
transaction_date    0
transaction_time    0
transaction_qty     0
store_id            0
store_location      0
product_id          0
unit_price          0
product_category    0
product_type        0
product_detail      0
dtype: int64

---

### Extracción de Características Temporales a partir de Fechas y Horas de Transacción

Se encarga de convertir las fechas y horas de transacción en un formato datetime y extraer características temporales relevantes. 

1. **Conversión a Formato Datetime**: Se crea una nueva columna `datetime` combinando las columnas `transaction_date` y `transaction_time`. Se utiliza `pd.to_datetime` para convertir esta combinación en un objeto datetime, especificando el formato correspondiente.

2. **Extracción de Características Temporales**:
   - **Día de la Semana**: Se añade una columna `day_of_week` que representa el día de la semana (0 para lunes y 6 para domingo) utilizando el método `.dt.weekday`.
   - **Hora**: Se añade una columna `hour` que extrae la hora (0-23) del objeto datetime.
   - **Mes**: Se añade una columna `month` que extrae el mes (1-12) del objeto datetime.

3. **Visualización de Resultados**: Se imprime la lista de columnas del DataFrame después de la extracción de características y se muestran las primeras filas del DataFrame con las columnas relevantes.

In [15]:
df['datetime'] = pd.to_datetime(
    df['transaction_date'].astype(str) + ' ' + df['transaction_time'].astype(str)
)

df['day_of_week'] = df['datetime'].dt.weekday
df['hour'] = df['datetime'].dt.hour
df['month'] = df['datetime'].dt.month

print('Columnas después de la extracción de características de fecha y hora', df.columns.tolist())
df[['transaction_date', 'transaction_time', 'datetime', 'day_of_week', 'hour', 'month']].head()

Columnas después de la extracción de características de fecha y hora ['transaction_id', 'transaction_date', 'transaction_time', 'transaction_qty', 'store_id', 'store_location', 'product_id', 'unit_price', 'product_category', 'product_type', 'product_detail', 'sales', 'datetime', 'day_of_week', 'hour', 'month']


,transaction_date,transaction_time,datetime,day_of_week,hour,month
0,2023-01-01,07:06:11,2023-01-01 07:06:11,6,7,1
1,2023-01-01,07:08:56,2023-01-01 07:08:56,6,7,1
2,2023-01-01,07:14:04,2023-01-01 07:14:04,6,7,1
3,2023-01-01,07:20:24,2023-01-01 07:20:24,6,7,1
4,2023-01-01,07:22:41,2023-01-01 07:22:41,6,7,1


---

### Análisis de Ventas por Hora y Ubicación de Tienda

Realiza un análisis de las ventas agrupadas por hora y ubicación de la tienda a partir de un DataFrame `df`.

1. **Cálculo de Ventas Totales**: Se crea una nueva columna `sales` que multiplica la cantidad de transacciones (`transaction_qty`) por el precio unitario (`unit_price`).

2. **Agrupación de Datos**: Se agrupan los datos por hora y ubicación de la tienda. Se utiliza `pd.Grouper(freq='H')` para agrupar por hora y se calcula la suma de las ventas totales.

3. **Renombrar Columnas**: Las columnas del DataFrame agrupado se renombran para mayor claridad, cambiando `datetime` a `datetime_hour` y `sales` a `total_sales_hour`.

4. **Extracción de Características Temporales**: Se añaden columnas adicionales que representan el día de la semana, la hora y el mes a partir de la columna `datetime_hour`.

5. **One-Hot Encoding**: Se aplica one-hot encoding a la columna `store_location` para convertir las ubicaciones de las tiendas en variables dummy, facilitando su uso en modelos de machine learning.

6. **Visualización de Resultados**: Finalmente, se imprime la lista de columnas del DataFrame resultante y se muestran las primeras filas del mismo.

In [19]:
df['sales'] = df['transaction_qty'] * df['unit_price']

df_grouped = (
    df
    .copy()
    .set_index('datetime')
    .groupby([pd.Grouper(freq='h'), 'store_location'])
    .agg({'sales': 'sum'})
    .reset_index()
)

df_grouped = df_grouped.rename(columns={
    'datetime': 'datetime_hour',
    'sales': 'total_sales_hour'
})

df_grouped['day_of_week'] = df_grouped['datetime_hour'].dt.weekday
df_grouped['hour'] = df_grouped['datetime_hour'].dt.hour
df_grouped['month'] = df_grouped['datetime_hour'].dt.month

df_with_dummies = pd.get_dummies(df_grouped, columns=['store_location'], drop_first=True)

print("Columnas despues de agrupar y del one-hot encoding:", df_with_dummies.columns.tolist())
df_with_dummies.head()

Columnas despues de agrupar y del one-hot encoding: ['datetime_hour', 'total_sales_hour', 'day_of_week', 'hour', 'month', "store_location_Hell's Kitchen", 'store_location_Lower Manhattan']


,datetime_hour,total_sales_hour,day_of_week,hour,month,store_location_Hell's Kitchen,store_location_Lower Manhattan
0,2023-01-01 07:00:00,81.05,6,7,1,False,True
1,2023-01-01 08:00:00,43.30,6,8,1,True,False
2,2023-01-01 08:00:00,42.60,6,8,1,False,True
3,2023-01-01 09:00:00,83.85,6,9,1,True,False
4,2023-01-01 09:00:00,102.50,6,9,1,False,True


---

### Predicción de Ventas Horarias de Café

Creamos un modelo que predice las ventas de café por hora usando datos de ventas. Vamos a usar un modelo de regresión lineal para hacer esto.

1. Definir el Objetivo y las Características: definimos qué queremos predecir (nuestras ventas totales por hora) y qué datos vamos a usar para hacer esa predicción.

2. Preparamos los Datos: separamos nuestros datos en dos partes: una para entrenar el modelo y otra para probarlo.

3. Crear y Entrenar el Modelo: creamos el modelo de regresión lineal y entrenarlo con los datos de entrenamiento.

4. Hacer Predicciones: Usamos nuestro modelo para hacer predicciones sobre los datos de entrenamiento y prueba.

6. Evaluar el Modelo: Probamos qué tan bien está funcionando nuestro modelo usando algunas métricas.

7. Guardar el Modelo: Guardamos nuestro modelo entrenado para poder usarlo más tarde.


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import joblib

target_col = "total_sales_hour"
feature_cols = ["day_of_week", "hour", "month"] + [
    col
    for col in df_with_dummies.columns
    if col.startswith("store_location_")
]

X = df_with_dummies[feature_cols]
y = df_with_dummies[target_col]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print(f"Train MSE: {mse_train:.2f}, R²: {r2_train:.3f}")
print(f"Test  MSE: {mse_test:.2f}, R²: {r2_test:.3f}")

joblib.dump(model, "coffee_sales_hour_model.pkl")

Train MSE: 3601.74, R²: 0.348
Test  MSE: 3745.11, R²: 0.318


['coffee_sales_hour_model.pkl']